# Capstone project - Toronto Neighborhoods

### This  notebook is for build the code, to scrape and clean up Wikipedia page 

In this notebook I will to explore, segment, and cluster the neighborhoods in the city of Toronto. 

Firs, I let's download all the dependencies that we will need.

In [62]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

I used BeautifulSoup library to scrap the Wikipedia page. Then, I transforme the data in format requiered, cleaning data remove the Borough whit Not assigned and missings, and finally obtein a DataFrame.

In [63]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bs = BeautifulSoup(page,'xml')

In [64]:
table = bs.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [65]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [66]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[(~df['PostalCode'].isnull()) & (df['Borough'] != 'Not assigned')]
df = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(list) 
df = pd.DataFrame(df)
df1 = df.sort_values(by=['Neighbourhood'])
df1 = df1.reset_index()

In [81]:
column_names = ['Neighbours'] 
neights = pd.DataFrame(columns=column_names)

prueba = df1.loc[:,'Neighbourhood']

for row in prueba:
    prueba1 = ','.join(row)
    neights = neights.append({'Neighbours' : prueba1}, ignore_index = True)

In [82]:
torn_venues = df1.drop(['Neighbourhood'], axis= 1)
trt_venues = pd.merge(torn_venues, neights, left_index=True, right_index=True)
trt_venues.head()

,PostalCode,Borough,Neighbours
0,M5H,Downtown Toronto,"Adelaide,King,Richmond"
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel..."
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
4,M8W,Etobicoke,"Alderwood,Long Branch"


#### Load and explore the data

Next, let's load the data, latitude and longitude.

In [12]:
df1.shape

(103, 3)

In [13]:
path ='http://cocl.us/Geospatial_data'
dfl = pd.read_csv(path)
dfl.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
dfc = pd.merge(trt_venues, dfl, left_on ='PostalCode', right_on = 'Postal Code')
dfc.head()

,PostalCode,Borough,vecis,Postal Code,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide,King,Richmond",M5H,43.650571,-79.384568
1,M1S,Scarborough,Agincourt,M1S,43.794200,-79.262029
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",M1V,43.815252,-79.284577
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V,43.739416,-79.588437
4,M8W,Etobicoke,"Alderwood,Long Branch",M8W,43.602414,-79.543484
